# 基于截断策略的机器阅读理解任务实战  

## 导入相关包                    
由于做的是类似 **问答任务** 故采用 `AutoModelForQuestionAnswering`

In [1]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

## 数据加载  

In [2]:
datasets = load_dataset("cmrc2018", cache_dir="data", trust_remote_code=True)
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [3]:
datasets["train"][0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

## 数据预处理  

In [4]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [5]:
sample_dataset = datasets["train"].select(range(10))

### tokenizer中的参数解释
1. `text`: 这个参数用于传递要分词的主要文本。在这个例子中，它来自于`sample_dataset["question"]`，这通常意味着它是问题文本。
2. `text_pair`: 这个参数用于传递与主要文本配对的第二段文本。在这个例子中，它来自于`sample_dataset["context"]`，这通常意味着它是与问题相关的上下文文本。在某些任务中，比如问答任务，模型的输入通常包括问题和其上下文。
3. `return_offsets_mapping`: 这个参数设置为`True`，意味着分词器将返回每个分词与其在原始文本中的位置映射。这对于后续的任务，比如解释模型预测或错误分析，是非常有用的。
4. `max_length`: 这个参数指定了分词器输出的序列的最大长度。在这个例子中，最大长度被设置为512个令牌（token）。如果输入文本的长度超过这个值，文本将被截断。
5. `truncation`: 这个参数指定了如何处理超出`max_length`的文本。在这个例子中，`truncation="only_second"` 意味着只有第二段文本（即`text_pair`）会被截断，而主要文本（`text`）不会被截断。


比如 `return_offsets_mapping = True`中，假设句子为：Hello, how are you?  得到的结果可能如下：
tokens: ['Hello', ',', 'how', 'are', 'you', '?']
offset_mapping: [(0, 5), (5, 6), (7, 10), (11, 14), (15, 18), (18, 19)]
    

In [6]:
tokenized_examples = tokenizer(text=sample_dataset["question"],
                               text_pair=sample_dataset["context"],
                               return_offsets_mapping=True,
                               max_length=512, truncation="only_second", padding="max_length")
tokenized_examples.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])

### 可以看到有两个 offset_mapping     
第二行第三个就是第二段文本的 (0,1)

In [7]:
print(tokenized_examples["offset_mapping"][0], "---", len(tokenized_examples["offset_mapping"][0]))

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [8]:
offset_mapping = tokenized_examples.pop("offset_mapping")

In [9]:
print(len(offset_mapping))

10


In [10]:
temp1 = tokenized_examples.sequence_ids(2)

In [11]:
print(temp1)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### 调试 answers 起始位置和结束位置(字符)     
这里拿到的仅仅只是字符的位置，后续还需要拿到token的位置才是我们想要的

In [12]:
for idx, offset in enumerate(offset_mapping):
    answer = sample_dataset[idx]['answers']
    start_char = answer['answer_start'][0]
    end_char = start_char + len(answer['text'][0])
    print(answer, start_char, end_char)

{'text': ['1963年'], 'answer_start': [30]} 30 35
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126
{'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598
{'text': ['范廷颂于2009年2月22日清晨在河内离世'], 'answer_start': [759]} 759 780
{'text': ['《全美超级模特儿新秀大赛》第十季'], 'answer_start': [26]} 26 42
{'text': ['有前途的新面孔'], 'answer_start': [247]} 247 254
{'text': ['《Jet》、《东方日报》、《Elle》等'], 'answer_start': [706]} 706 726
{'text': ['售货员'], 'answer_start': [202]} 202 205
{'text': ['大空翼'], 'answer_start': [84]} 84 87


### 代码解释     
首先，`tokenized_examples.sequence_ids(idx)`返回的是一个列表，其中包含了当前样本（由索引`idx`指定）在分词后的序列中的每个令牌的类型。这个列表通常包含三个可能的值：





- `0`：表示该令牌来自`question`文本。     
- `1`：表示该令牌来自`context`文本。     
- `None`：表示该令牌是特殊令牌，如分隔符或填充符。     
例如，假设我们有一个分词后的序列，其`sequence_ids`可能如下所示：     
```
[0, 0, 0, 0, None, 1, 1, 1, 1, 1, None, None, None]





```
在这个例子中，前四个令牌来自问题文本，然后是一个特殊令牌（如分隔符），接下来的五个令牌来自上下文文本，最后是三个填充令牌。





现在，让我们解释这两句代码：     
1. `context_start = tokenized_examples.sequence_ids(idx).index(1)`：这行代码查找`sequence_ids`列表中第一个值为`1`的令牌的位置。这个位置标志着`context`文本在分词后序列中的开始。在上面的例子中，`context_start`将是5。





2. `context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1`：这行代码从`context_start`开始查找第一个值为`None`的令牌的位置，然后减去1，得到`context`文本在分词后序列中的结束位置。在上面的例子中，`context_end`将是9。





至于`if`判断中的`offset`，它来自于之前提到的`offset_mapping`，这是一个由分词器返回的列表，其中包含了每个分词在原始文本中的起始和结束位置。在`if`判断中，`offset`用于比较答案的字符位置和`context`文本的字符位置，以确定答案是否在`context`文本中。






In [13]:
for idx, offset in enumerate(offset_mapping):
    answer = sample_dataset[idx]["answers"]
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer["text"][0])
    # 定位答案在token中的起始位置和结束位置  
    # 一种策略，我们要拿到context的起始和结束，然后从左右两侧向答案逼近  

    context_start = tokenized_examples.sequence_ids(idx).index(1)
    context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1

    # 判断答案是否在context中
    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        start_token_pos = 0
        end_token_pos = 0
    else:
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -=1
        end_token_pos = token_id
        
    print(answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenized_examples["input_ids"][idx][start_token_pos: end_token_pos + 1]))

{'text': ['1963年'], 'answer_start': [30]} 30 35 17 510 47 48
token answer decode: 1963 年
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 510 53 70
token answer decode: 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 510 100 124
token answer decode: 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生
{'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598 17 510 0 0
token answer decode: [CLS]
{'text': ['范廷颂于2009年2月22日清晨在河内离世'], 'answer_start': [759]} 759 780 12 510 0 0
token answer decode: [CLS]
{'text': ['《全美超级模特儿新秀大赛》第十季'], 'answer_start': [26]} 26 42 21 510 47 62
token answer decode: 《 全 美 超 级 模 特 儿 新 秀 大 赛 》 第 十 季
{'text': ['有前途的新面孔'], 'answer_start': [247]} 247 254 20 510 232 238
token answer decode: 有 前 途 的 新 面 孔
{'text': ['《Jet》、《东方日报》、《Elle》等'], 'answer_start': [706]} 706 726 20 510 0 0
token answer decode: [CLS]
{'text': ['售货员'], 'answer_start': [202]}

## 将上面调试好的代码进行封装

In [15]:
def process_func(examples):
    tokenized_examples = tokenizer(text=examples["question"],
                               text_pair=examples["context"],
                               return_offsets_mapping=True,
                               max_length=384, truncation="only_second", padding="max_length")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for idx, offset in enumerate(offset_mapping):
        answer = examples["answers"][idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        # 定位答案在token中的起始位置和结束位置
        # 一种策略，我们要拿到context的起始和结束，然后从左右两侧向答案逼近
        context_start = tokenized_examples.sequence_ids(idx).index(1)
        context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1
        # 判断答案是否在context中
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -=1
            end_token_pos = token_id
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
    
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

In [17]:
tokenied_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)

tokenied_datasets

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1002
    })
})

## 4️⃣ model loading    

In [18]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base", trust_remote_code=True)

E:\software\anaconda3\envs\transformers\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 5️⃣ 配置 TrainingArguments 

In [19]:
args = TrainingArguments(
    output_dir="models_for_mrc",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=1
)

E:\software\anaconda3\envs\transformers\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
train_args = TrainingArguments(output_dir="models_for_mrc",      # 输出文件夹
                               per_device_train_batch_size=1,   # 训练时的batch_size
                               gradient_accumulation_steps=32,  # *** 梯度累加 ***
                               gradient_checkpointing=True,     # *** 梯度检查点 ***
                               optim="adafactor",               # *** adafactor优化器 *** 
                               per_device_eval_batch_size=1,    # 验证时的batch_size
                               num_train_epochs=1,              # 训练轮数
                               logging_steps=10,                # log 打印的频率
                               eval_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型

## 6️⃣ 配置Trainer  

In [21]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenied_datasets["train"],
    eval_dataset=tokenied_datasets["validation"],
    data_collator=DefaultDataCollator()
)

## 7️⃣ model training   
3070 8G 训练了 1个小时9分钟

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.382400,1.101883


TrainOutput(global_step=317, training_loss=1.7628894589300788, metrics={'train_runtime': 4181.6163, 'train_samples_per_second': 2.425, 'train_steps_per_second': 0.076, 'total_flos': 1987553780112384.0, 'train_loss': 1.7628894589300788, 'epoch': 1.0})

## 8️⃣ model prediction  

In [23]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

In [24]:
pipe(question="小明在哪里上班？", context="小明在北京上班。")

{'score': 0.6652287840843201, 'start': 3, 'end': 5, 'answer': '北京'}